In [4]:
import xgboost as xbg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import joblib

%matplotlib inline

In [6]:
def load_cls(fname):
    return np.array(pickle.load(open(fname,"rb")))
path = "/scratch/pbanerj6/sml-dataset/"

X_train_cls = load_cls(path+"X_train_cls.p")
X_val_cls = load_cls(path+"X_val_cls.p")


In [7]:
y_train = load_cls(path+"y_train.p")
y_val = load_cls(path+"y_val.p")

In [25]:
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

#model = XGBClassifier(n_jobs=-1,verbosity=2)
model = DecisionTreeClassifier(random_state=0)
from sklearn.model_selection import cross_val_score
model.fit(X_train_cls,y_train)
# cross_val_score(model, X_train_cls, y_train, cv=10)
# print(cross_val_score)

In [ ]:
#model.fit(X_train_cls,y_train,verbose=True,eval_set=[(X_val_cls,y_val)])

In [29]:
from sklearn.metrics import accuracy_score

In [30]:
preds_val = model.predict(X_val_cls)

In [31]:
accuracy = accuracy_score(y_val, preds_val)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 53.72%


In [32]:
X_test_cls = load_cls(path+"X_test_cls.p")
y_test = load_cls(path+"y_test.p")

In [33]:
preds_test = model.predict(X_test_cls)
accuracy = accuracy_score(y_test, preds_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 52.60%


In [34]:
X_test_cls[0].shape[0]

768

In [35]:
from scipy.stats import rankdata

def mrrs(out, labels):
#     print(out,labels)
    outputs = np.argmax(out,axis=1)
    mrr = 0.0 
    for label,ranks in zip(labels,out):
        ranks = rankdata(ranks*-1)
        rank = ranks[label]
#         print(rank,ranks)
        mrr+=1/rank
    return mrr/len(labels)

def mrrwrapper(qid2c,qid2indexmap,preds_prob):
    labels = []
    out = []
    for qid in qid2c.keys():
        scores = []
        for ix in qid2indexmap[qid]:
            if len(scores) < 6:
                scores.append(preds_prob[ix][1])
        if len(scores) < 6:
            continue
        out.append(scores)
        labels.append(int(qid2c[qid]))
    return mrrs(np.array(out),labels)

def load_ranking(fname):
    return pickle.load(open(path+"ranking_"+fname+".p","rb"))

In [36]:
preds_test_probs = model.predict_proba(X_test_cls)

In [37]:
qid2c,qid2indexmap = load_ranking("test")

In [38]:
mrrwrapper(qid2c,qid2indexmap,preds_test_probs)

0.3632683982683975